## quick_pp

In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()
all_data = project.get_all_data()

# Rock Typing

The process of rock typing is a multidisciplinary effort that involves close collaboration between geologists, petrophysicists, and reservoir engineers. Each discipline plays a specific role, and the rock typing workflow involves a seamless transition of data, models, and insights across these domains to build a robust understanding of the reservoir. Below is the high-level workflow detailing the steps, key tasks, and handoffs between these professionals

Petrophysicists take geological inputs and combine them with well log data and core analysis to define rock types based on petrophysical properties such as porosity, permeability, fluid saturation, and pore geometry.

Tasks:
- Core Data Integration:
    - Perform routine and special core analyses (RCAL, SCAL) to measure porosity, permeability, capillary pressure, and wettability.

- Log Interpretation:
    - Interpret wireline logs (e.g., density, neutron, resistivity, NMR) to generate continuous petrophysical properties along the wellbore.

- Rock Typing Methods:
    - Use statistical tools like Flow Zone Indicator (FZI), Pickett plots, and cluster analysis to group rocks with similar flow behavior into petrophysical rock types.

- Electrofacies Analysis:
    - Identify electrofacies by clustering log responses, especially where core data is limited.

The rock typing workflow is a collaborative process where geologists, petrophysicists, and reservoir engineers work together to ensure the reservoir is accurately characterized and modeled. This interdisciplinary approach ensures that both geological and petrophysical complexities are accounted for, resulting in more efficient production strategies and better field development plans

## Winland R35 Rock Typing

In [ ]:
from quick_pp.rock_type import plot_fzi, plot_rfn, plot_winland, rock_typing
from quick_pp.rock_type import calc_r35, rock_typing, plot_winland

all_data['CPERM'] = all_data.CORE_PERM
all_data['CPORE'] = all_data.CORE_POR / 100
# all_data['CPERM'] = np.where(all_data.CPERM < .01, np.nan, all_data.CPERM)
# all_data['CPORE'] = np.where(all_data.CPORE < .01, np.nan, all_data.CPORE)

# Estimate rock types
r35 = calc_r35(all_data['CPORE'], all_data['CPERM'])
all_data['R35'] = r35
r35_cut_offs = [
    .1, .15, .2, .25, .3, .35, .4, .5, .6, .7, .8, .9, 1, 1.25, 1.5, 2, 2.5, 3, 4, 5, 7, 10,
]
r35_rock_flag = rock_typing(r35, r35_cut_offs, higher_is_better=True)
all_data['ROCK_FLAG'] = r35_rock_flag

plot_winland(all_data['CPORE'], all_data['CPERM'], rock_type=r35_rock_flag, cut_offs=r35_cut_offs)
print('The data has been classified into the following rock types:')
print(all_data['ROCK_FLAG'].value_counts())

### Develop Machine Learning models to predict ROCK_FLAG and R35

In [ ]:
from quick_pp.rock_type import train_classification_model, train_regression_model

train_data = all_data.copy()  # Filter out low porosity data
# train_data = train_data[train_data.CPORE >= .02].copy()  # Filter out low porosity data
# train_data = train_data[train_data.CPERM > .1]  # Filter out low permeability data
# train_data = train_data[train_data.ROCK_FLAG < 5]  # Filter out non-reservoir data

train_data['LOG_RT'] = np.log10(train_data['RT'])
train_data['NDI_V2'] = np.log10(((2.85 - train_data['RHOB']) / 1.85) - train_data['NPHI'])
input_features = ['GR', 'NPHI', 'RHOB', 'LOG_RT', 'NDI_V2']  
train_data = train_data.dropna(subset=input_features + ['ROCK_FLAG', 'R35'])

r35_rt_model = train_classification_model(
    train_data, input_features=input_features, target_feature='ROCK_FLAG')
with open(r'data\04_project\MOCK_carbonate\outputs\r35_rt_model.qppm', 'wb') as file:
    pickle.dump(r35_rt_model, file)

train_data['LOG_R35'] = np.log10(train_data['R35'])
r35_model = train_regression_model(
    train_data, input_features=input_features, target_feature='LOG_R35', stratifier=train_data['ROCK_FLAG'])
with open(r'data\04_project\MOCK_carbonate\outputs\r35_model.qppm', 'wb') as file:
    pickle.dump(r35_model, file)

### Determining the perm transform parameter for each rock type.

In [ ]:
from ipywidgets import interact, widgets
from quick_pp.core_calibration import poroperm_xplot, fit_poroperm_curve

rt = widgets.Dropdown(
    options=all_data['ROCK_FLAG'].dropna().unique(),
    value=1,
    description='Rock Type:'
)

@interact(rt=rt)
def param(rt):
    data = all_data[all_data['ROCK_FLAG'] == rt]
    a, b = fit_poroperm_curve(data['CPORE'], data['CPERM'])
    poroperm_xplot(data['CPORE'], data['CPERM'], a=a, b=b, label=rt, log_log=False)

In [ ]:
import pprint

poroperm_params = {}

for rt, data in all_data.groupby('ROCK_FLAG'):
    a, b = fit_poroperm_curve(data['CPORE'], data['CPERM'])
    poroperm_params[rt] = (a, b)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(poroperm_params)

In [ ]:
for rt, data in all_data.groupby('ROCK_FLAG'):
    a, b = poroperm_params[rt]
    poroperm_xplot(data['CPORE'], data['CPERM'], a=a, b=b, label=f'RT{rt}: a={a:1d}, b={b:.2f}')

### Compare different PERM estimations at well level

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score

from quick_pp.core_calibration import perm_transform
from quick_pp.rock_type import calc_r35_perm

focus_well = 'HW-26'
well_data = all_data[all_data.WELL_NAME == focus_well].copy()

# Permeability estimation based on core data perm transform
well_data['perm_a'] = well_data['ROCK_FLAG'].map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
well_data['perm_b'] = well_data['ROCK_FLAG'].map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_trans = perm_transform(well_data['PHIT'], well_data['perm_a'], well_data['perm_b'])
perm_r35 = calc_r35_perm(well_data['R35'], well_data['PHIT'])

# Permeability prediction based on ROCK_FLAG ML model followed by perm transform
with open(r'data\04_project\MOCK_carbonate\outputs\r35_rt_model.qppm', 'rb') as file:
    r35_rt_model = pickle.load(file)
rock_flag_ml = r35_rt_model.predict(well_data[input_features])
perm_a_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_ml = perm_transform(well_data['PHIT'], perm_a_ml, perm_b_ml)

# Permeability prediction based on R35 ML model followed by back calculate from R35
with open(r'data\04_project\MOCK_carbonate\outputs\r35_model.qppm', 'rb') as file:
    r35_model = pickle.load(file)
r35_ml = 10**(r35_model.predict(well_data[input_features]))
perm_r35_ml = calc_r35_perm(r35_ml, well_data['PHIT'])

# Plot to compare
plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, perm_trans, label='Perm Transform')
plt.plot(well_data.DEPTH, perm_r35, label='Perm R35')
plt.plot(well_data.DEPTH, perm_ml, label='Perm ML')
plt.plot(well_data.DEPTH, perm_r35_ml, label='Perm R35 ML')
plt.scatter(well_data.DEPTH, well_data.CPERM, label='Core Perm', marker='.', c='black')
plt.yscale('log')
plt.legend()

# Print scores
score_df = well_data[['DEPTH', 'CPERM']].copy()
score_df['PERM'] = perm_r35_ml
score_df.dropna(inplace=True)
print(f"\n ### PERM MAPE: {mean_absolute_percentage_error(score_df.CPERM, score_df.PERM):.2f}")
print(f" ### PERM R2: {r2_score(score_df.CPERM, score_df.PERM):.2f}")

### Compare different ROCK_FLAG at well level

In [ ]:
# Compare rock types predicted from rt_model with applied cut-offs on predicted r35
rock_flag_r35_ml = rock_typing(r35_ml, r35_cut_offs, higher_is_better=True)

plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, well_data.ROCK_FLAG, label='Rock Flag Core')
plt.plot(well_data.DEPTH, rock_flag_ml, label='Rock Flag ML')
plt.plot(well_data.DEPTH, rock_flag_r35_ml, label='Rock Flag R35 ML')
plt.legend()


## Plotting the Rock Types

As a comparison, the rock types being identified using FZI is plotted on Winland R35 and Lucia Rock Fabric Number (RFN) methods.

- The plot demonstrates Winland R35 resulted in a more flat permeability estimation while
- Lower porosity - high permeability datapoints.
    - These points might be indicating fracture kind of rock types
    - Both Winland R35 and Lucia RFN does not model the datapoints
    - FZI is able to model but indicates too high of a value

In [ ]:
plot_fzi(all_data['CPORE'], all_data['CPERM'], rock_type=r35_rock_flag)
plot_winland(all_data['CPORE'], all_data['CPERM'], rock_type=r35_rock_flag)
plot_rfn(all_data['CPORE'], all_data['CPERM'], rock_type=r35_rock_flag)

# Plotting the result

In [ ]:
from quick_pp.plotter import plotly_log

# Plot individual results
well_data['PERM'] = perm_r35_ml
fig = plotly_log(well_data, 'ft')
fig.show(config=dict(scrollZoom=True))

In [ ]:
# # Save the well data
# project.update_data(well_data)
# project.save()